In [ ]:
from pyspark.sql import SparkSession, functions as F
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

import matplotlib.pyplot as plt#
import pandas as pd
# importações 

In [4]:
# Cria SparkSession 
spark = (
    SparkSession.builder
    .appName("TraseBrazilBeef-ML")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)

spark


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/29 21:02:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/29 21:02:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/29 21:02:14 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
BUCKET = "dados-supplychain"
BASE_PATH = f"s3a://{BUCKET}/trase/brazil_beef"
SILVER_STATE_PATH = f"{BASE_PATH}/silver/flows_by_state"

df_state = spark.read.parquet(SILVER_STATE_PATH)

df_state.printSchema()
df_state.select(
    "year",
    "state_of_production",
    "total_volume_t",
    "total_cattle_defor_ha",
    "total_co2_gross_tco2"
).show(10, truncate=False)


25/11/29 21:02:28 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


root
 |-- year: integer (nullable = true)
 |-- state_of_production: string (nullable = true)
 |-- state_of_production_trase_id: string (nullable = true)
 |-- total_volume_t: double (nullable = true)
 |-- total_fob_usd: double (nullable = true)
 |-- total_cattle_defor_ha: double (nullable = true)
 |-- total_co2_gross_tco2: double (nullable = true)
 |-- total_co2_net_tco2: double (nullable = true)
 |-- total_pasture_area_ha: double (nullable = true)
 |-- avg_forest500_beef: double (nullable = true)
 |-- sum_zdc_flag: double (nullable = true)
 |-- zdc_volume_t: double (nullable = true)
 |-- zdc_share_volume: double (nullable = true)



+----+-------------------+------------------+---------------------+--------------------+
|year|state_of_production|total_volume_t    |total_cattle_defor_ha|total_co2_gross_tco2|
+----+-------------------+------------------+---------------------+--------------------+
|2020|TOCANTINS          |94935.75596838303 |113251.47184709366   |1.8215550543871008E7|
|2020|PARA               |171823.01349184045|116706.75844987041   |6.203819009127608E7 |
|2020|SAO PAULO          |466634.7965043159 |511.8384858391535    |179493.36307918283  |
|2017|ESPIRITO SANTO     |6576.719122505074 |390.62208684050574   |162484.3996349078   |
|2020|PARANA             |17456.427153614262|120.95889784138247   |50856.65012591888   |
|2023|TOCANTINS          |111007.5952635722 |87997.82798924933    |1.360351769937709E7 |
|2020|MARANHAO           |5468.455430119726 |4069.7946418945994   |1139182.83533762    |
|2020|RONDONIA           |188693.9138590178 |80293.18621830289    |3.6471042166826844E7|
|2017|RIO GRANDE DO S

In [6]:
# Garante tipo numérico
df_state = df_state.withColumn(
    "total_cattle_defor_ha",
    F.col("total_cattle_defor_ha").cast("double")
)

# Calcula mediana aproximada de desmatamento
median_defor = df_state.approxQuantile("total_cattle_defor_ha", [0.5], 0.01)[0]
print("Mediana de desmatamento (ha):", median_defor)

# Cria coluna label
df_ml = df_state.withColumn(
    "label",
    F.when(F.col("total_cattle_defor_ha") >= median_defor, 1).otherwise(0)
)

df_ml.select(
    "year",
    "state_of_production",
    "total_cattle_defor_ha",
    "label"
).orderBy("total_cattle_defor_ha", ascending=False).show(10, truncate=False)


Mediana de desmatamento (ha): 766.9759772173004
+----+-------------------+---------------------+-----+
|year|state_of_production|total_cattle_defor_ha|label|
+----+-------------------+---------------------+-----+
|2023|UNKNOWN STATE      |6867003.686805465    |1    |
|2022|UNKNOWN STATE      |5660588.954519147    |1    |
|2019|UNKNOWN STATE      |5106863.1082406035   |1    |
|2021|UNKNOWN STATE      |5039183.704432643    |1    |
|2020|UNKNOWN STATE      |4934323.666656295    |1    |
|2017|UNKNOWN STATE      |4395232.150147333    |1    |
|2016|UNKNOWN STATE      |4333790.607647422    |1    |
|2015|UNKNOWN STATE      |4302173.113117231    |1    |
|2022|MATO GROSSO        |196154.1681612758    |1    |
|2023|MATO GROSSO        |179559.48038375698   |1    |
+----+-------------------+---------------------+-----+
only showing top 10 rows



In [7]:
# Garante tipos numéricos das features
num_features = [
    "total_volume_t",
    "total_fob_usd",
    "total_pasture_area_ha",
    "total_co2_gross_tco2",
    "avg_forest500_beef",
    "zdc_share_volume"
]

for col_name in num_features:
    if col_name in df_ml.columns:
        df_ml = df_ml.withColumn(col_name, F.col(col_name).cast("double"))

# Index da variável categórica 'state_of_production'
state_indexer = StringIndexer(
    inputCol="state_of_production",
    outputCol="state_index"
)

# Monta vetor de features
assembler = VectorAssembler(
    inputCols=num_features + ["state_index"],
    outputCol="features"
)

# Modelo de classificação
rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="label",
    numTrees=200,
    maxDepth=8,
    seed=42
)

# Pipeline completo
pipeline = Pipeline(stages=[state_indexer, assembler, rf])

# Treino / teste
train_df, test_df = df_ml.randomSplit([0.8, 0.2], seed=42)

print("Tamanho treino:", train_df.count())
print("Tamanho teste:", test_df.count())


Tamanho treino: 190
Tamanho teste: 35


In [9]:
feature_cols = [
    "total_volume_t",
    "total_fob_usd",
    "total_pasture_area_ha",
    "total_co2_gross_tco2",
    "avg_forest500_beef",
    "zdc_share_volume",
]

print("Contagem de null por coluna numérica:")
for c in feature_cols:
    df_ml.select(
        F.count(F.when(F.col(c).isNull(), 1)).alias(c)
    ).show()

# também checar estado e label
df_ml.select(
    F.count(F.when(F.col("state_of_production").isNull(), 1)).alias("state_null"),
    F.count(F.when(F.col("label").isNull(), 1)).alias("label_null")
).show()


Contagem de null por coluna numérica:


+--------------+
|total_volume_t|
+--------------+
|             0|
+--------------+

+-------------+
|total_fob_usd|
+-------------+
|            0|
+-------------+

+---------------------+
|total_pasture_area_ha|
+---------------------+
|                   17|
+---------------------+

+--------------------+
|total_co2_gross_tco2|
+--------------------+
|                  67|
+--------------------+

+------------------+
|avg_forest500_beef|
+------------------+
|                76|
+------------------+

+----------------+
|zdc_share_volume|
+----------------+
|               0|
+----------------+

+----------+----------+
|state_null|label_null|
+----------+----------+
|         0|         0|
+----------+----------+



In [10]:
# Colunas numéricas que vão para o modelo
num_features = [
    "total_volume_t",
    "total_fob_usd",
    "total_pasture_area_ha",
    "total_co2_gross_tco2",
    "avg_forest500_beef",
    "zdc_share_volume"
]

# garantir tipos numéricos
for col_name in num_features:
    if col_name in df_ml.columns:
        df_ml = df_ml.withColumn(col_name, F.col(col_name).cast("double"))

# remover linhas com null em qualquer feature ou na categórica
cols_sem_null = num_features + ["state_of_production", "label"]

df_ml_clean = df_ml.na.drop(subset=cols_sem_null)

print("Antes:", df_ml.count(), "linhas")
print("Depois de limpar nulls:", df_ml_clean.count(), "linhas")


Antes: 225 linhas
Depois de limpar nulls: 134 linhas


In [12]:
model = pipeline.fit(train_df)

preds = model.transform(test_df)

preds.select(
    "state_of_production",
    "year",
    "label",
    "prediction",
    "probability"
).show(20, truncate=False)


25/11/29 21:05:04 ERROR Executor: Exception in task 0.0 in stage 57.0 (TID 40)
org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda$3933/0x00000008418a3840`: (struct<total_volume_t:double,total_fob_usd:double,total_pasture_area_ha:double,total_co2_gross_tco2:double,avg_forest500_beef:double,zdc_share_volume:double,state_index:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFa

Py4JJavaError: An error occurred while calling o108.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 57.0 failed 1 times, most recent failure: Lost task 0.0 in stage 57.0 (TID 40) (1ca62cf5066f executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda$3933/0x00000008418a3840`: (struct<total_volume_t:double,total_fob_usd:double,total_pasture_area_ha:double,total_co2_gross_tco2:double,avg_forest500_beef:double,zdc_share_volume:double,state_index:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNe

In [ ]:
# AUC (área sob curva ROC)
bin_eval = BinaryClassificationEvaluator(
    labelCol="label",
    rawPredictionCol="rawPrediction"
)

auc = bin_eval.evaluate(preds)
print("AUC:", auc)

# Acurácia
acc_eval = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)

accuracy = acc_eval.evaluate(preds)
print("Acurácia:", accuracy)


In [ ]:
# Último estágio do pipeline é o RandomForest
rf_model = model.stages[-1]

feature_list = num_features + ["state_index"]
importances = rf_model.featureImportances.toArray()

imp_df = pd.DataFrame({
    "feature": feature_list,
    "importance": importances
}).sort_values("importance", ascending=False)

imp_df


In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(imp_df["feature"], imp_df["importance"])
plt.xticks(rotation=45, ha="right")
plt.title("Importância das variáveis – Random Forest (exposição ao desmatamento)")
plt.ylabel("Importância relativa")
plt.tight_layout()
plt.show()


In [ ]:
ML_BASE = f"{BASE_PATH}/gold/ml_exports_by_state"

(preds
 .select(
     "year",
     "state_of_production",
     "state_of_production_trase_id",
     "total_volume_t",
     "total_cattle_defor_ha",
     "label",
     "prediction",
     "probability"
 )
 .write
 .mode("overwrite")
 .parquet(ML_BASE)
)

print(f"Resultados de ML salvos em: {ML_BASE}")
